In [1]:
from __future__ import annotations

import pickle
from pathlib import Path
from timeit import default_timer

import numpy as np
from tqdm import tqdm
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset
import logging
import h5py

In [2]:
class GatingTransformer(nn.Module):
    """
    Transforms the original input to match target resolution and channels
    for use in the gating mechanism at different layers
    """
    def __init__(self, in_channels, out_channels, target_size):
        super(GatingTransformer, self).__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.target_size = target_size

        # Transform input channels to match layer's channel requirement
        self.channel_transform = nn.Conv1d(
            in_channels=in_channels,
            out_channels=out_channels,
            kernel_size=1
        )

    def forward(self, x):
        # Resize input to target resolution
        x = F.interpolate(x, size=self.target_size, mode='linear', align_corners=False)
        # Transform channels
        x = self.channel_transform(x)
        return x

class DLGN_Gate(nn.Module):
    """
    Implements the gating mechanism for DLGN
    G_l(x) = σ(βU_l^T x)
    """
    def __init__(self, in_channels, out_channels, target_size, beta=1.0):
        super(DLGN_Gate, self).__init__()
        self.beta = beta
        self.target_size = target_size
        self.transformer = GatingTransformer(in_channels, out_channels, target_size)
        self.gate_weights = nn.Conv1d(
            in_channels=out_channels,
            out_channels=out_channels,
            kernel_size=3,
            padding=1
        )

    def forward(self, original_input):
        # Transform original input to match current layer dimensions
        x = self.transformer(original_input)
        # Apply gating weights (U_l^T x)
        x = self.gate_weights(x)
        # Apply sigmoid activation with beta parameter
        return torch.sigmoid(self.beta * x)

class DLGN_Layer(nn.Module):
    """
    Implements the DLGN activation mechanism
    ψ^(l)(x) = G_l(x) ⊙ (W_l^T ψ^(l-1)(x))
    """
    def __init__(self, in_size, out_size):
        super(DLGN_Layer, self).__init__()
        self.in_size = in_size
        self.out_size = out_size

    def forward(self, x, gate):
        # Resize input to desired output size (double size first as in original CNO_LReLu)
        x = F.interpolate(x.unsqueeze(2), size=(1, 2 * self.in_size), mode="bicubic", antialias=True)

        # Make sure gate has the right shape for broadcasting
        if gate.shape != x.shape:
            # Reshape gate to match the interpolated x
            gate = gate.unsqueeze(2) if gate.dim() == 3 else gate
            gate = F.interpolate(gate, size=x.size()[2:], mode="nearest")

        # Element-wise multiplication with gate
        x = gate * x

        # Resize to final output size
        x = F.interpolate(x, size=(1, self.out_size), mode="bicubic", antialias=True)
        return x[:,:,0]

In [3]:
class CNOBlock_DLGN(nn.Module):
    """
    Modified CNOBlock to use DLGN instead of traditional activation
    """
    def __init__(self,
                 in_channels,
                 out_channels,
                 in_size,
                 out_size,
                 original_channels,
                 beta=1.0,
                 use_bn=True):
        super(CNOBlock_DLGN, self).__init__()

        self.in_channels = in_channels
        self.out_channels = out_channels
        self.in_size = in_size
        self.out_size = out_size

        # Convolution layer
        self.convolution = torch.nn.Conv1d(in_channels=self.in_channels,
                                          out_channels=self.out_channels,
                                          kernel_size=3,
                                          padding=1)

        # Batch normalization (optional)
        if use_bn:
            self.batch_norm = nn.BatchNorm1d(self.out_channels)
        else:
            self.batch_norm = nn.Identity()

        # DLGN components
        self.gating = DLGN_Gate(original_channels, out_channels, out_size, beta)
        self.dlgn_layer = DLGN_Layer(in_size=self.in_size, out_size=self.out_size)

    def forward(self, x, original_input):
        # Apply convolution and batch norm
        x = self.convolution(x)
        x = self.batch_norm(x)

        # Get gating signal from original input
        gate = self.gating(original_input)

        # Apply DLGN layer
        return self.dlgn_layer(x, gate)

class LiftProjectBlock_DLGN(nn.Module):
    """
    Modified LiftProjectBlock to use DLGN
    """
    def __init__(self,
                in_channels,
                out_channels,
                size,
                original_channels,
                beta=1.0,
                latent_dim=64):
        super(LiftProjectBlock_DLGN, self).__init__()

        self.inter_CNOBlock = CNOBlock_DLGN(in_channels=in_channels,
                                           out_channels=latent_dim,
                                           in_size=size,
                                           out_size=size,
                                           original_channels=original_channels,
                                           beta=beta,
                                           use_bn=False)

        self.convolution = torch.nn.Conv1d(in_channels=latent_dim,
                                          out_channels=out_channels,
                                          kernel_size=3,
                                          padding=1)

    def forward(self, x, original_input):
        x = self.inter_CNOBlock(x, original_input)
        x = self.convolution(x)
        return x

class ResidualBlock_DLGN(nn.Module):
    """
    Modified ResidualBlock to use DLGN
    """
    def __init__(self,
                 channels,
                 size,
                 original_channels,
                 beta=1.0,
                 use_bn=True):
        super(ResidualBlock_DLGN, self).__init__()

        self.channels = channels
        self.size = size

        # First convolution path
        self.convolution1 = torch.nn.Conv1d(in_channels=self.channels,
                                           out_channels=self.channels,
                                           kernel_size=3,
                                           padding=1)

        # Second convolution path
        self.convolution2 = torch.nn.Conv1d(in_channels=self.channels,
                                           out_channels=self.channels,
                                           kernel_size=3,
                                           padding=1)

        # Batch normalization (optional)
        if use_bn:
            self.batch_norm1 = nn.BatchNorm1d(self.channels)
            self.batch_norm2 = nn.BatchNorm1d(self.channels)
        else:
            self.batch_norm1 = nn.Identity()
            self.batch_norm2 = nn.Identity()

        # DLGN components
        self.gating1 = DLGN_Gate(original_channels, channels, size, beta)
        self.gating2 = DLGN_Gate(original_channels, channels, size, beta)
        self.dlgn_layer1 = DLGN_Layer(in_size=self.size, out_size=self.size)
        self.dlgn_layer2 = DLGN_Layer(in_size=self.size, out_size=self.size)

    def forward(self, x, original_input):
        identity = x

        # First convolution path
        out = self.convolution1(x)
        out = self.batch_norm1(out)

        # Apply first DLGN layer
        gate1 = self.gating1(original_input)
        out = self.dlgn_layer1(out, gate1)

        # Second convolution path
        out = self.convolution2(out)
        out = self.batch_norm2(out)

        # Apply second DLGN layer
        gate2 = self.gating2(original_input)
        out = self.dlgn_layer2(out, gate2)

        # Apply residual connection
        return identity + out

class ResNet_DLGN(nn.Module):
    """
    Modified ResNet to use DLGN
    """
    def __init__(self,
                 channels,
                 size,
                 original_channels,
                 num_blocks,
                 beta=1.0,
                 use_bn=True):
        super(ResNet_DLGN, self).__init__()

        self.channels = channels
        self.size = size
        self.num_blocks = num_blocks
        self.original_channels = original_channels

        # Create resnet blocks
        self.res_blocks = nn.ModuleList([
            ResidualBlock_DLGN(
                channels=channels,
                size=size,
                original_channels=original_channels,
                beta=beta,
                use_bn=use_bn
            ) for _ in range(num_blocks)
        ])

    def forward(self, x, original_input):
        for block in self.res_blocks:
            x = block(x, original_input)
        return x

In [4]:
class CNO1d(nn.Module):
    def __init__(self,
                 num_channels,           # Number of input/output channels
                 width=64,               # Base width for the network
                 initial_step=10,        # Number of initial timesteps
                 size=None,              # Spatial size of the domain
                 N_layers=4,             # Number of (D) or (U) blocks in the network
                 N_res=4,                # Number of (R) blocks per level (except the neck)
                 N_res_neck=4,           # Number of (R) blocks in the neck
                 channel_multiplier=16,  # How the number of channels evolve
                 use_bn=True             # Add BN? We do not add BN in lifting/projection layer
                ):
        super(CNO1d, self).__init__()

        # Store parameters
        self.num_channels = num_channels
        self.initial_step = initial_step
        self.width = width

        # Determine spatial size if not provided
        if size is None:
            size = 128  # Default spatial size

        # Create the CNO1d architecture
        self.model = CNO1d_DLGN(
            in_dim=initial_step * num_channels + 1,  # Input: initial_step channels + grid
            out_dim=num_channels,                    # Output: predict next channel values
            size=size,
            N_layers=N_layers,
            N_res=N_res,
            N_res_neck=N_res_neck,
            channel_multiplier=channel_multiplier,
            use_bn=use_bn
        )

    def forward(self, x, grid):
        # x dim = [b, x1, t*v] - same input format as FNO1d
        # Combine x and grid to match CNO1d_Core input format
        inputs = torch.cat((x, grid), dim=-1)  # concatenate along feature dimension

        # Reshape for CNO: [batch, channels, spatial_dim]
        batch_size = inputs.shape[0]
        spatial_dim = inputs.shape[1]
        inputs = inputs.permute(0, 2, 1)  # [batch, features, spatial]

        # Forward pass through CNO
        out = self.model(inputs)

        # Reshape back to expected output format: [batch, spatial, channels, 1]
        out = out.permute(0, 2, 1)  # [batch, spatial, channels]
        out = out.unsqueeze(-2)     # [batch, spatial, 1, channels]

        return out

class CNO1d_DLGN(nn.Module):
    """
    Modified CNO1d to use DLGN framework
    """
    def __init__(self,
                 in_dim,                    # Number of input channels
                 out_dim,                   # Number of output channels
                 size,                      # Input and Output spatial size
                 N_layers,                  # Number of (D) or (U) blocks
                 N_res=4,                   # Number of (R) blocks per level
                 N_res_neck=4,              # Number of (R) blocks in the neck
                 channel_multiplier=16,     # Channel evolution factor
                 beta=1.0,                  # Beta parameter for DLGN
                 use_bn=True,               # Use batch normalization
                ):
        super(CNO1d_DLGN, self).__init__()

        self.N_layers = int(N_layers)
        self.lift_dim = channel_multiplier//2
        self.in_dim = in_dim
        self.out_dim = out_dim
        self.channel_multiplier = channel_multiplier
        self.beta = beta

        ######## Num of channels/features - evolution ########
        self.encoder_features = [self.lift_dim]
        for i in range(self.N_layers):
            self.encoder_features.append(2 ** i * self.channel_multiplier)

        self.decoder_features_in = self.encoder_features[1:]
        self.decoder_features_in.reverse()
        self.decoder_features_out = self.encoder_features[:-1]
        self.decoder_features_out.reverse()

        for i in range(1, self.N_layers):
            self.decoder_features_in[i] = 2 * self.decoder_features_in[i]

        ######## Spatial sizes of channels - evolution ########
        self.encoder_sizes = []
        self.decoder_sizes = []
        for i in range(self.N_layers + 1):
            self.encoder_sizes.append(size // 2 ** i)
            self.decoder_sizes.append(size // 2 ** (self.N_layers - i))

        ######## Define Lift and Project blocks ########
        self.lift = LiftProjectBlock_DLGN(
            in_channels=in_dim,
            out_channels=self.encoder_features[0],
            size=size,
            original_channels=in_dim,
            beta=beta
        )

        self.project = LiftProjectBlock_DLGN(
            in_channels=self.encoder_features[0] + self.decoder_features_out[-1],
            out_channels=out_dim,
            size=size,
            original_channels=in_dim,
            beta=beta
        )

        ######## Define Encoder, ED Linker and Decoder networks ########
        self.encoder = nn.ModuleList([
            CNOBlock_DLGN(
                in_channels=self.encoder_features[i],
                out_channels=self.encoder_features[i+1],
                in_size=self.encoder_sizes[i],
                out_size=self.encoder_sizes[i+1],
                original_channels=in_dim,
                beta=beta,
                use_bn=use_bn
            ) for i in range(self.N_layers)
        ])

        # ED expansion blocks
        self.ED_expansion = nn.ModuleList([
            CNOBlock_DLGN(
                in_channels=self.encoder_features[i],
                out_channels=self.encoder_features[i],
                in_size=self.encoder_sizes[i],
                out_size=self.decoder_sizes[self.N_layers - i],
                original_channels=in_dim,
                beta=beta,
                use_bn=use_bn
            ) for i in range(self.N_layers + 1)
        ])

        # Decoder blocks
        self.decoder = nn.ModuleList([
            CNOBlock_DLGN(
                in_channels=self.decoder_features_in[i],
                out_channels=self.decoder_features_out[i],
                in_size=self.decoder_sizes[i],
                out_size=self.decoder_sizes[i+1],
                original_channels=in_dim,
                beta=beta,
                use_bn=use_bn
            ) for i in range(self.N_layers)
        ])

        ######## Define ResNets Blocks ########
        self.res_nets = nn.ModuleList([
            ResNet_DLGN(
                channels=self.encoder_features[l],
                size=self.encoder_sizes[l],
                original_channels=in_dim,
                num_blocks=N_res,
                beta=beta,
                use_bn=use_bn
            ) for l in range(self.N_layers)
        ])

        # Bottleneck ResNet
        self.res_net_neck = ResNet_DLGN(
            channels=self.encoder_features[self.N_layers],
            size=self.encoder_sizes[self.N_layers],
            original_channels=in_dim,
            num_blocks=N_res_neck,
            beta=beta,
            use_bn=use_bn
        )

    def forward(self, x):
        # Store original input for gating
        original_input = x.clone()

        # Execute lift
        x = self.lift(x, original_input)
        skip = []

        # Execute encoder
        for i in range(self.N_layers):
            # Apply ResNet & save result
            y = self.res_nets[i](x, original_input)
            skip.append(y)

            # Apply (D) block
            x = self.encoder[i](x, original_input)

        # Apply bottleneck ResNet
        x = self.res_net_neck(x, original_input)

        # Execute decoder
        for i in range(self.N_layers):
            # Apply (I) block (ED_expansion) & cat if needed
            if i == 0:
                x = self.ED_expansion[self.N_layers - i](x, original_input)  # Bottleneck: no cat
            else:
                exp_skip = self.ED_expansion[self.N_layers - i](skip[-i], original_input)
                x = torch.cat((x, exp_skip), 1)

            # Apply (U) block
            x = self.decoder[i](x, original_input)

        # Cat & execute projection
        exp_skip = self.ED_expansion[0](skip[0], original_input)
        x = torch.cat((x, exp_skip), 1)
        x = self.project(x, original_input)

        return x

In [5]:
class FNODatasetSingle(Dataset):
    def __init__(
        self,
        filename,
        initial_step=10,
        saved_folder="../data/",
        reduced_resolution=1,
        reduced_resolution_t=1,
        reduced_batch=1,
        if_test=False,
        test_ratio=0.1,
        num_samples_max=-1,
    ):
        """

        :param filename: filename that contains the dataset
        :type filename: STR
        :param filenum: array containing indices of filename included in the dataset
        :type filenum: ARRAY
        :param initial_step: time steps taken as initial condition, defaults to 10
        :type initial_step: INT, optional

        """

        # Define path to files
        root_path = Path(Path(saved_folder).resolve()) / filename
        if filename[-2:] != "h5":
            # print(".HDF5 file extension is assumed hereafter")

            with h5py.File(root_path, "r") as f:
                keys = list(f.keys())
                keys.sort()
                if "tensor" not in keys:
                    _data = np.array(
                        f["density"], dtype=np.float32
                    )  # batch, time, x,...
                    idx_cfd = _data.shape
                    if len(idx_cfd) == 3:  # 1D
                        self.data = np.zeros(
                            [
                                idx_cfd[0] // reduced_batch,
                                idx_cfd[2] // reduced_resolution,
                                mt.ceil(idx_cfd[1] / reduced_resolution_t),
                                3,
                            ],
                            dtype=np.float32,
                        )
                        # density
                        _data = _data[
                            ::reduced_batch,
                            ::reduced_resolution_t,
                            ::reduced_resolution,
                        ]
                        ## convert to [x1, ..., xd, t, v]
                        _data = np.transpose(_data[:, :, :], (0, 2, 1))
                        self.data[..., 0] = _data  # batch, x, t, ch
                        # pressure
                        _data = np.array(
                            f["pressure"], dtype=np.float32
                        )  # batch, time, x,...
                        _data = _data[
                            ::reduced_batch,
                            ::reduced_resolution_t,
                            ::reduced_resolution,
                        ]
                        ## convert to [x1, ..., xd, t, v]
                        _data = np.transpose(_data[:, :, :], (0, 2, 1))
                        self.data[..., 1] = _data  # batch, x, t, ch
                        # Vx
                        _data = np.array(
                            f["Vx"], dtype=np.float32
                        )  # batch, time, x,...
                        _data = _data[
                            ::reduced_batch,
                            ::reduced_resolution_t,
                            ::reduced_resolution,
                        ]
                        ## convert to [x1, ..., xd, t, v]
                        _data = np.transpose(_data[:, :, :], (0, 2, 1))
                        self.data[..., 2] = _data  # batch, x, t, ch

                        self.grid = np.array(f["x-coordinate"], dtype=np.float32)
                        self.grid = torch.tensor(
                            self.grid[::reduced_resolution], dtype=torch.float
                        ).unsqueeze(-1)
                        # print(self.data.shape)
                    if len(idx_cfd) == 4:  # 2D
                        self.data = np.zeros(
                            [
                                idx_cfd[0] // reduced_batch,
                                idx_cfd[2] // reduced_resolution,
                                idx_cfd[3] // reduced_resolution,
                                mt.ceil(idx_cfd[1] / reduced_resolution_t),
                                4,
                            ],
                            dtype=np.float32,
                        )
                        # density
                        _data = _data[
                            ::reduced_batch,
                            ::reduced_resolution_t,
                            ::reduced_resolution,
                            ::reduced_resolution,
                        ]
                        ## convert to [x1, ..., xd, t, v]
                        _data = np.transpose(_data, (0, 2, 3, 1))
                        self.data[..., 0] = _data  # batch, x, t, ch
                        # pressure
                        _data = np.array(
                            f["pressure"], dtype=np.float32
                        )  # batch, time, x,...
                        _data = _data[
                            ::reduced_batch,
                            ::reduced_resolution_t,
                            ::reduced_resolution,
                            ::reduced_resolution,
                        ]
                        ## convert to [x1, ..., xd, t, v]
                        _data = np.transpose(_data, (0, 2, 3, 1))
                        self.data[..., 1] = _data  # batch, x, t, ch
                        # Vx
                        _data = np.array(
                            f["Vx"], dtype=np.float32
                        )  # batch, time, x,...
                        _data = _data[
                            ::reduced_batch,
                            ::reduced_resolution_t,
                            ::reduced_resolution,
                            ::reduced_resolution,
                        ]
                        ## convert to [x1, ..., xd, t, v]
                        _data = np.transpose(_data, (0, 2, 3, 1))
                        self.data[..., 2] = _data  # batch, x, t, ch
                        # Vy
                        _data = np.array(
                            f["Vy"], dtype=np.float32
                        )  # batch, time, x,...
                        _data = _data[
                            ::reduced_batch,
                            ::reduced_resolution_t,
                            ::reduced_resolution,
                            ::reduced_resolution,
                        ]
                        ## convert to [x1, ..., xd, t, v]
                        _data = np.transpose(_data, (0, 2, 3, 1))
                        self.data[..., 3] = _data  # batch, x, t, ch

                        x = np.array(f["x-coordinate"], dtype=np.float32)
                        y = np.array(f["y-coordinate"], dtype=np.float32)
                        x = torch.tensor(x, dtype=torch.float)
                        y = torch.tensor(y, dtype=torch.float)
                        X, Y = torch.meshgrid(x, y, indexing="ij")
                        self.grid = torch.stack((X, Y), axis=-1)[
                            ::reduced_resolution, ::reduced_resolution
                        ]

                    if len(idx_cfd) == 5:  # 3D
                        self.data = np.zeros(
                            [
                                idx_cfd[0] // reduced_batch,
                                idx_cfd[2] // reduced_resolution,
                                idx_cfd[3] // reduced_resolution,
                                idx_cfd[4] // reduced_resolution,
                                mt.ceil(idx_cfd[1] / reduced_resolution_t),
                                5,
                            ],
                            dtype=np.float32,
                        )
                        # density
                        _data = _data[
                            ::reduced_batch,
                            ::reduced_resolution_t,
                            ::reduced_resolution,
                            ::reduced_resolution,
                            ::reduced_resolution,
                        ]
                        ## convert to [x1, ..., xd, t, v]
                        _data = np.transpose(_data, (0, 2, 3, 4, 1))
                        self.data[..., 0] = _data  # batch, x, t, ch
                        # pressure
                        _data = np.array(
                            f["pressure"], dtype=np.float32
                        )  # batch, time, x,...
                        _data = _data[
                            ::reduced_batch,
                            ::reduced_resolution_t,
                            ::reduced_resolution,
                            ::reduced_resolution,
                            ::reduced_resolution,
                        ]
                        ## convert to [x1, ..., xd, t, v]
                        _data = np.transpose(_data, (0, 2, 3, 4, 1))
                        self.data[..., 1] = _data  # batch, x, t, ch
                        # Vx
                        _data = np.array(
                            f["Vx"], dtype=np.float32
                        )  # batch, time, x,...
                        _data = _data[
                            ::reduced_batch,
                            ::reduced_resolution_t,
                            ::reduced_resolution,
                            ::reduced_resolution,
                            ::reduced_resolution,
                        ]
                        ## convert to [x1, ..., xd, t, v]
                        _data = np.transpose(_data, (0, 2, 3, 4, 1))
                        self.data[..., 2] = _data  # batch, x, t, ch
                        # Vy
                        _data = np.array(
                            f["Vy"], dtype=np.float32
                        )  # batch, time, x,...
                        _data = _data[
                            ::reduced_batch,
                            ::reduced_resolution_t,
                            ::reduced_resolution,
                            ::reduced_resolution,
                            ::reduced_resolution,
                        ]
                        ## convert to [x1, ..., xd, t, v]
                        _data = np.transpose(_data, (0, 2, 3, 4, 1))
                        self.data[..., 3] = _data  # batch, x, t, ch
                        # Vz
                        _data = np.array(
                            f["Vz"], dtype=np.float32
                        )  # batch, time, x,...
                        _data = _data[
                            ::reduced_batch,
                            ::reduced_resolution_t,
                            ::reduced_resolution,
                            ::reduced_resolution,
                            ::reduced_resolution,
                        ]
                        ## convert to [x1, ..., xd, t, v]
                        _data = np.transpose(_data, (0, 2, 3, 4, 1))
                        self.data[..., 4] = _data  # batch, x, t, ch

                        x = np.array(f["x-coordinate"], dtype=np.float32)
                        y = np.array(f["y-coordinate"], dtype=np.float32)
                        z = np.array(f["z-coordinate"], dtype=np.float32)
                        x = torch.tensor(x, dtype=torch.float)
                        y = torch.tensor(y, dtype=torch.float)
                        z = torch.tensor(z, dtype=torch.float)
                        X, Y, Z = torch.meshgrid(x, y, z, indexing="ij")
                        self.grid = torch.stack((X, Y, Z), axis=-1)[
                            ::reduced_resolution,
                            ::reduced_resolution,
                            ::reduced_resolution,
                        ]

                else:  # scalar equations
                    ## data dim = [t, x1, ..., xd, v]
                    _data = np.array(
                        f["tensor"], dtype=np.float32
                    )  # batch, time, x,...
                    if len(_data.shape) == 3:  # 1D
                        _data = _data[
                            ::reduced_batch,
                            ::reduced_resolution_t,
                            ::reduced_resolution,
                        ]
                        ## convert to [x1, ..., xd, t, v]
                        _data = np.transpose(_data[:, :, :], (0, 2, 1))
                        self.data = _data[:, :, :, None]  # batch, x, t, ch

                        self.grid = np.array(f["x-coordinate"], dtype=np.float32)
                        self.grid = torch.tensor(
                            self.grid[::reduced_resolution], dtype=torch.float
                        ).unsqueeze(-1)
                    if len(_data.shape) == 4:  # 2D Darcy flow
                        # u: label
                        _data = _data[
                            ::reduced_batch,
                            :,
                            ::reduced_resolution,
                            ::reduced_resolution,
                        ]
                        ## convert to [x1, ..., xd, t, v]
                        _data = np.transpose(_data[:, :, :, :], (0, 2, 3, 1))
                        # if _data.shape[-1]==1:  # if nt==1
                        #    _data = np.tile(_data, (1, 1, 1, 2))
                        self.data = _data
                        # nu: input
                        _data = np.array(
                            f["nu"], dtype=np.float32
                        )  # batch, time, x,...
                        _data = _data[
                            ::reduced_batch,
                            None,
                            ::reduced_resolution,
                            ::reduced_resolution,
                        ]
                        ## convert to [x1, ..., xd, t, v]
                        _data = np.transpose(_data[:, :, :, :], (0, 2, 3, 1))
                        self.data = np.concatenate([_data, self.data], axis=-1)
                        self.data = self.data[:, :, :, :, None]  # batch, x, y, t, ch

                        x = np.array(f["x-coordinate"], dtype=np.float32)
                        y = np.array(f["y-coordinate"], dtype=np.float32)
                        x = torch.tensor(x, dtype=torch.float)
                        y = torch.tensor(y, dtype=torch.float)
                        X, Y = torch.meshgrid(x, y, indexing="ij")
                        self.grid = torch.stack((X, Y), axis=-1)[
                            ::reduced_resolution, ::reduced_resolution
                        ]

        elif filename[-2:] == "h5":  # SWE-2D (RDB)
            # print(".H5 file extension is assumed hereafter")

            with h5py.File(root_path, "r") as f:
                keys = list(f.keys())
                keys.sort()

                data_arrays = [
                    np.array(f[key]["data"], dtype=np.float32) for key in keys
                ]
                _data = torch.from_numpy(
                    np.stack(data_arrays, axis=0)
                )  # [batch, nt, nx, ny, nc]
                _data = _data[
                    ::reduced_batch,
                    ::reduced_resolution_t,
                    ::reduced_resolution,
                    ::reduced_resolution,
                    ...,
                ]
                _data = torch.permute(_data, (0, 2, 3, 1, 4))  # [batch, nx, ny, nt, nc]
                gridx, gridy = (
                    np.array(f["0023"]["grid"]["x"], dtype=np.float32),
                    np.array(f["0023"]["grid"]["y"], dtype=np.float32),
                )
                mgridX, mgridY = np.meshgrid(gridx, gridy, indexing="ij")
                _grid = torch.stack(
                    (torch.from_numpy(mgridX), torch.from_numpy(mgridY)), axis=-1
                )
                _grid = _grid[::reduced_resolution, ::reduced_resolution, ...]
                _tsteps_t = torch.from_numpy(
                    np.array(f["0023"]["grid"]["t"], dtype=np.float32)
                )

                tsteps_t = _tsteps_t[::reduced_resolution_t]
                self.data = _data
                self.grid = _grid
                self.tsteps_t = tsteps_t

        if num_samples_max > 0:
            num_samples_max = min(num_samples_max, self.data.shape[0])
        else:
            num_samples_max = self.data.shape[0]

        test_idx = int(num_samples_max * test_ratio)
        if if_test:
            self.data = self.data[:test_idx]
        else:
            self.data = self.data[test_idx:num_samples_max]

        # Time steps used as initial conditions
        self.initial_step = initial_step

        self.data = self.data if torch.is_tensor(self.data) else torch.tensor(self.data)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx, ..., : self.initial_step, :], self.data[idx], self.grid

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
logger = logging.getLogger(__name__)

In [7]:
filename = "1D_Advection_Sols_beta0.1_reduced.hdf5"
base_path = "/content/drive/MyDrive/"

# Training settings
if_training = True
continue_training = False
num_workers = 0
batch_size = 50
initial_step = 10
t_train = 200
epochs = 500
learning_rate = 1e-3
scheduler_step = 100
scheduler_gamma = 0.5
model_update = 1

In [8]:
num_channels = 1
size = 128  # Spatial grid size (important for CNO)

# New CNO specific parameters (replacing FNO parameters)
N_layers = 4
N_res = 4
N_res_neck = 4
channel_multiplier = 16

# Dataset preprocessing options
single_file = True
reduced_resolution = 1
reduced_resolution_t = 1
reduced_batch = 1

# Plotting and bounds
plot = False
training_type = "autoregressive"

In [9]:
model_name = filename[:-5] + "_CNO_75"
model_path = model_name + ".pt"

In [10]:
model_path

'1D_Advection_Sols_beta0.1_reduced_CNO_75.pt'

In [11]:
train_data = FNODatasetSingle(
    filename,
    reduced_resolution=reduced_resolution,
    reduced_resolution_t=reduced_resolution_t,
    reduced_batch=reduced_batch,
    initial_step=initial_step,
    saved_folder=base_path,
)

val_data = FNODatasetSingle(
    filename,
    reduced_resolution=reduced_resolution,
    reduced_resolution_t=reduced_resolution_t,
    reduced_batch=reduced_batch,
    initial_step=initial_step,
    if_test=True,
    saved_folder=base_path,
)

In [12]:
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, num_workers=num_workers, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_data, batch_size=batch_size, num_workers=num_workers, shuffle=False)

In [13]:
print("NUmber of batches",len(train_loader))
for batch in train_loader:
    print(f"Batch type: {type(batch)}")
    if isinstance(batch, (tuple, list)):
        print("Shapes of elements in batch:")
        for item in batch:
            print(item.shape)
    else:
        print(batch.shape)
    break  # only show the first batch

NUmber of batches 180
Batch type: <class 'list'>
Shapes of elements in batch:
torch.Size([50, 256, 10, 1])
torch.Size([50, 256, 41, 1])
torch.Size([50, 256, 1])


In [14]:
_, _data, _grid = next(iter(val_loader))
t_train = min(t_train, _data.shape[-2])
size = _data.shape[1]

In [15]:
model = CNO1d(
    num_channels=num_channels,
    width=channel_multiplier,  # Use channel_multiplier as width
    initial_step=initial_step,
    size=size,
    N_layers=N_layers,
    N_res=N_res,
    N_res_neck=N_res_neck,
    channel_multiplier=channel_multiplier,
    use_bn=True
).to(device)

In [16]:
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=scheduler_step, gamma=scheduler_gamma)
loss_fn = nn.MSELoss(reduction="mean")
loss_val_min = np.inf
start_epoch = 0

In [17]:
def metric_func(
    pred, target, if_mean=True, Lx=1.0, Ly=1.0, Lz=1.0, iLow=4, iHigh=12, initial_step=1
):
    """
    code for calculate metrics discussed in the Brain-storming session
    RMSE, normalized RMSE, max error, RMSE at the boundaries, conserved variables, RMSE in Fourier space, temporal sensitivity
    """
    pred, target = pred.to(device), target.to(device)
    # (batch, nx^i..., timesteps, nc)
    # slice out `initial context` timesteps
    pred = pred[..., initial_step:, :]
    target = target[..., initial_step:, :]
    idxs = target.size()
    if len(idxs) == 4:  # 1D
        pred = pred.permute(0, 3, 1, 2)
        target = target.permute(0, 3, 1, 2)

    idxs = target.size()
    nb, nc, nt = idxs[0], idxs[1], idxs[-1]

    # RMSE
    err_mean = torch.sqrt(
        torch.mean(
            (pred.view([nb, nc, -1, nt]) - target.view([nb, nc, -1, nt])) ** 2, dim=2
        )
    )
    err_RMSE = torch.mean(err_mean, axis=0)
    nrm = torch.sqrt(torch.mean(target.view([nb, nc, -1, nt]) ** 2, dim=2))
    err_nRMSE = torch.mean(err_mean / nrm, dim=0)

    err_CSV = torch.sqrt(
        torch.mean(
            (
                torch.sum(pred.view([nb, nc, -1, nt]), dim=2)
                - torch.sum(target.view([nb, nc, -1, nt]), dim=2)
            )
            ** 2,
            dim=0,
        )
    )
    if len(idxs) == 4:
        nx = idxs[2]
        err_CSV /= nx

    # worst case in all the data
    err_Max = torch.max(
        torch.max(
            torch.abs(pred.view([nb, nc, -1, nt]) - target.view([nb, nc, -1, nt])),
            dim=2,
        )[0],
        dim=0,
    )[0]

    if len(idxs) == 4:  # 1D
        err_BD = (pred[:, :, 0, :] - target[:, :, 0, :]) ** 2
        err_BD += (pred[:, :, -1, :] - target[:, :, -1, :]) ** 2
        err_BD = torch.mean(torch.sqrt(err_BD / 2.0), dim=0)

    if len(idxs) == 4:  # 1D
        nx = idxs[2]
        pred_F = torch.fft.rfft(pred, dim=2)
        target_F = torch.fft.rfft(target, dim=2)
        _err_F = (
            torch.sqrt(torch.mean(torch.abs(pred_F - target_F) ** 2, axis=0)) / nx * Lx
        )

    err_F = torch.zeros([nc, 3, nt]).to(device)
    err_F[:, 0] += torch.mean(_err_F[:, :iLow], dim=1)  # low freq
    err_F[:, 1] += torch.mean(_err_F[:, iLow:iHigh], dim=1)  # middle freq
    err_F[:, 2] += torch.mean(_err_F[:, iHigh:], dim=1)  # high freq

    if if_mean:
        return (
            torch.mean(err_RMSE, dim=[0, -1]),
            torch.mean(err_nRMSE, dim=[0, -1]),
            torch.mean(err_CSV, dim=[0, -1]),
            torch.mean(err_Max, dim=[0, -1]),
            torch.mean(err_BD, dim=[0, -1]),
            torch.mean(err_F, dim=[0, -1]),
        )
    return err_RMSE, err_nRMSE, err_CSV, err_Max, err_BD, err_F

In [18]:
def metrics(
    val_loader,
    model,
    Lx,
    Ly,
    Lz,
    plot,
    channel_plot,
    model_name,
    x_min,
    x_max,
    y_min,
    y_max,
    t_min,
    t_max,
    mode="CNO",  # Changed default to CNO
    initial_step=None,
):
    if mode == "CNO":  # Update to use CNO instead of FNO
        with torch.no_grad():
            itot = 0
            for itot, (xx, yy, grid) in enumerate(val_loader):
                xx = xx.to(device)  # noqa: PLW2901
                yy = yy.to(device)  # noqa: PLW2901
                grid = grid.to(device)  # noqa: PLW2901

                pred = yy[..., :initial_step, :]
                inp_shape = list(xx.shape)
                inp_shape = inp_shape[:-2]
                inp_shape.append(-1)

                for _t in range(initial_step, yy.shape[-2]):
                    inp = xx.reshape(inp_shape)
                    im = model(inp, grid)
                    pred = torch.cat((pred, im), -2)
                    xx = torch.cat((xx[..., 1:, :], im), dim=-2)  # noqa: PLW2901

                (
                    _err_RMSE,
                    _err_nRMSE,
                    _err_CSV,
                    _err_Max,
                    _err_BD,
                    _err_F,
                ) = metric_func(
                    pred,
                    yy,
                    if_mean=True,
                    Lx=Lx,
                    Ly=Ly,
                    Lz=Lz,
                    initial_step=initial_step,
                )
                if itot == 0:
                    err_RMSE, err_nRMSE, err_CSV, err_Max, err_BD, err_F = (
                        _err_RMSE,
                        _err_nRMSE,
                        _err_CSV,
                        _err_Max,
                        _err_BD,
                        _err_F,
                    )
                    pred_plot = pred[:1]
                    target_plot = yy[:1]
                    val_l2_time = torch.zeros(yy.shape[-2]).to(device)
                else:
                    err_RMSE += _err_RMSE
                    err_nRMSE += _err_nRMSE
                    err_CSV += _err_CSV
                    err_Max += _err_Max
                    err_BD += _err_BD
                    err_F += _err_F

                    mean_dim = list(range(len(yy.shape) - 2))
                    mean_dim.append(-1)
                    mean_dim = tuple(mean_dim)
                    val_l2_time += torch.sqrt(
                        torch.mean((pred - yy) ** 2, dim=mean_dim)
                    )

    err_RMSE = np.array(err_RMSE.data.cpu() / itot)
    err_nRMSE = np.array(err_nRMSE.data.cpu() / itot)
    err_CSV = np.array(err_CSV.data.cpu() / itot)
    err_Max = np.array(err_Max.data.cpu() / itot)
    err_BD = np.array(err_BD.data.cpu() / itot)
    err_F = np.array(err_F.data.cpu() / itot)
    logger.info(f"RMSE: {err_RMSE:.5f}")
    logger.info(f"normalized RMSE: {err_nRMSE:.5f}")
    logger.info(f"RMSE of conserved variables: {err_CSV:.5f}")
    logger.info(f"Maximum value of rms error: {err_Max:.5f}")
    logger.info(f"RMSE at boundaries: {err_BD:.5f}")
    logger.info(f"RMSE in Fourier space: {err_F}")

    val_l2_time = val_l2_time / itot

    return err_RMSE, err_nRMSE, err_CSV, err_Max, err_BD, err_F

In [19]:
checkpoint = torch.load(model_path, map_location=device)

model.load_state_dict(checkpoint["model_state_dict"])
optimizer.load_state_dict(checkpoint["optimizer_state_dict"])
start_epoch = checkpoint["epoch"]
loss_val_min = checkpoint["loss"]

In [20]:
model.to(device)
model.eval()

CNO1d(
  (model): CNO1d_DLGN(
    (lift): LiftProjectBlock_DLGN(
      (inter_CNOBlock): CNOBlock_DLGN(
        (convolution): Conv1d(11, 64, kernel_size=(3,), stride=(1,), padding=(1,))
        (batch_norm): Identity()
        (gating): DLGN_Gate(
          (transformer): GatingTransformer(
            (channel_transform): Conv1d(11, 64, kernel_size=(1,), stride=(1,))
          )
          (gate_weights): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,))
        )
        (dlgn_layer): DLGN_Layer()
      )
      (convolution): Conv1d(64, 8, kernel_size=(3,), stride=(1,), padding=(1,))
    )
    (project): LiftProjectBlock_DLGN(
      (inter_CNOBlock): CNOBlock_DLGN(
        (convolution): Conv1d(16, 64, kernel_size=(3,), stride=(1,), padding=(1,))
        (batch_norm): Identity()
        (gating): DLGN_Gate(
          (transformer): GatingTransformer(
            (channel_transform): Conv1d(11, 64, kernel_size=(1,), stride=(1,))
          )
          (gate_weights): Conv1d(6

In [21]:
all_preds_val = []
all_truth_val = []

In [22]:
with torch.no_grad():
    for xx, yy, grid in tqdm(val_loader):  # or use test_loader if available
        xx, yy, grid = xx.to(device), yy.to(device), grid.to(device)
        pred = yy[..., :initial_step, :]
        inp_shape = list(xx.shape[:-2]) + [-1]

        for t in range(initial_step, yy.shape[-2]):
            inp = xx.reshape(inp_shape)
            im = model(inp, grid)
            pred = torch.cat((pred, im), -2)
            xx = torch.cat((xx[..., 1:, :], im), dim=-2)

        all_preds_val.append(pred.cpu().numpy())
        all_truth_val.append(yy[..., :pred.shape[-2], :].cpu().numpy())

100%|██████████| 20/20 [00:18<00:00,  1.07it/s]


In [23]:
sample = val_loader.dataset[0]
print("Sample keys / structure:")
if isinstance(sample, dict):
    print(sample.keys())
else:
    print(type(sample))
    if isinstance(sample, tuple):
        for i, s in enumerate(sample):
            print(f"Element {i}: shape {getattr(s, 'shape', type(s))}")

Sample keys / structure:
<class 'tuple'>
Element 0: shape torch.Size([256, 10, 1])
Element 1: shape torch.Size([256, 41, 1])
Element 2: shape torch.Size([256, 1])


In [24]:
predictions = np.concatenate(all_preds_val, axis=0)  # shape: (n_samples, n_t, n_x)
ground_truth = np.concatenate(all_truth_val, axis=0)

In [25]:
xx, yy, grid = next(iter(val_loader))
x = np.sort(np.unique(grid[:, 0].cpu().numpy()))  # shape: (n_x,)
t = np.linspace(0, 1, yy.shape[-2])               # shape: (n_t,)

In [26]:
np.savez("predictions_for_sindy.npz", u_pred=predictions, u_true=ground_truth, x=x, t=t)
print("✅ Saved predictions and grids for SINDy: 'predictions_for_sindy.npz'")

✅ Saved predictions and grids for SINDy: 'predictions_for_sindy.npz'


In [1]:
pip install pysindy numpy==1.24.0

In [3]:
u_pred.shape

(1000, 256, 41, 1)

In [4]:
xL = 1.0  # Left boundary
xR = 0.0  # Right boundary
nx = u_pred.shape[1]  # Extract the number of spatial points (from prediction shape)

dx = (xR - xL) / nx
xe = np.linspace(xL, xR, nx + 1)  # Edge points
x = xe[:-1] + 0.5 * dx  # Cell centers (excluding the last edge)

# Now x should be a 1D array with shape (nx,)
print(x.shape)

(256,)


In [5]:
t.shape

(41,)

In [20]:
import numpy as np
import pysindy as ps

# ---- Load saved predictions ----
data = np.load("predictions_for_sindy.npz")
u_pred = data["u_true"]                # (n_x,)
t = data["t"]

In [21]:
u_pred.shape

(1000, 256, 41, 1)

In [22]:
t.shape

(41,)

In [23]:
x.shape

(256,)

In [24]:
u = u_pred[0]                 # shape: (n_t, n_x)

# ---- Set up grid spacing
dt = t[1] - t[0]
dx = x[1] - x[0]

In [25]:
fd = ps.FiniteDifference()

In [26]:
library = ps.PDELibrary(
    library_functions=[lambda u: u],
    function_names=[lambda s: s],
    derivative_order=3,
    spatial_grid=x,
    include_interaction=False,
    is_uniform=True,
    include_bias=False
)

In [27]:
model = ps.SINDy(
    optimizer=ps.STLSQ(alpha=1e-5, threshold=1e-3),
    feature_library=library,
    differentiation_method=fd
)

In [28]:
model.fit(u, t=dt)
model.print()

(x0)' = -0.156 x0_1


Test run with batch size 50
epoch: 0, loss: 0.63057, t2-t1: 463.56203, trainL2: 9.55097, testL2: 0.38045
epoch: 1, loss: 0.55014, t2-t1: 460.28001, trainL2: 4.16928, testL2: 0.31352
epoch: 2, loss: 0.13798, t2-t1: 459.41851, trainL2: 2.85387, testL2: 0.06545
epoch: 3, loss: 0.09339, t2-t1: 460.18633, trainL2: 1.86878, testL2: 0.07487
epoch: 4, loss: 0.04525, t2-t1: 462.79572, trainL2: 1.96279, testL2: 0.02735
epoch: 5, loss: 0.03533, t2-t1: 460.98556, trainL2: 1.58974, testL2: 0.02083
epoch: 6, loss: 0.06565, t2-t1: 460.80017, trainL2: 1.52914, testL2: 0.05132
epoch: 7, loss: 0.04068, t2-t1: 458.44843, trainL2: 1.72168, testL2: 0.02438
epoch: 8, loss: 0.04329, t2-t1: 459.96253, trainL2: 1.50155, testL2: 0.02675
epoch: 9, loss: 0.05211, t2-t1: 455.91932, trainL2: 1.49865, testL2: 0.03940